![Practicum AI Logo image](https://github.com/PracticumAI/practicumai.github.io/blob/main/images/logo/PracticumAI_logo_250x50.png?raw=true)
***
# *Practicum AI:* GAN - The Final Model

This exercise was adapted from Baig et al. (2020) <i> The Deep Learning Workshop</i> from <a href="https://www.packtpub.com/product/the-deep-learning-workshop/9781839219856">Packt Publishers</a> (Exercise 7.04, page 348).

In this exercise, we will build and train the GAN using the method described in the previous section.

To finish this exercise, follow these steps:

<div style="padding: 10px;margin-bottom: 20px;border: thin solid #30335D;border-left-width: 10px;background-color: #fff"><strong>Note:</strong> As this exercise builds on and extends exercise 7.03, these setup sections come from that earlier section. </div>


#### 1. Import the requisite libraries

In [ ]:
# Import the required library functions

import tensorflow as tf
import numpy as np

from numpy.random import randn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from matplotlib import pyplot

#### 2. Define a function to generate real samples
This function is equivalent to the one created in exercise **01.2_generate_data** that generates a data distribution.

In [2]:
def realData(loc, batch):
    '''Function to generate real samples. '''
    
    # loc is the random location or mean around which samples are centred
    # Generate numbers to right of the random point
    xr = np.arange(loc, loc + (0.1 * batch / 2), 0.1)
    xr = xr[0:int(batch / 2)]
    
    # Generate numbers to left of the random point
    xl = np.arange(loc - (0.1 * batch / 2), loc, 0.1)
    
    # Concatenating both these series 
    X1 = np.concatenate((xl, xr))
    
    # Second dependent variable
    X2 = np.sin(X1)
    
    # Reshaping both the variables and then concatenating them to an array of independent variables
    X1 = X1.reshape(batch, 1)
    X2 = X2.reshape(batch, 1)    
    X  = np.concatenate((X1, X2),axis = 1)
    
    # Generating the labels for the real data set which is 'ones'
    y = np.ones((batch, 1)) 
    
    return X,y

#### 3. Define a function to generate inputs for the generator network
This function generates a fake dataset sampled from the random distribution.

In [3]:
def fakeInputs(batch, infeats):
    '''Function to generate inputs for the generator function. '''
    
    # Sample data points equal to (batch x input feature size) from a random distribution
    genInput = randn(infeats * batch)
    
    # Reshape the input 
    X = genInput.reshape(batch ,infeats)
    
    return X

#### 4. Define a function to create the generator network
This function is the same as the one we built in exercise **01.3_generator** to create a generator network.

In [4]:
def genModel(infeats, outfeats):
    '''Function which instantiates the generator model. '''
    
    # Define and instantiate the Generator model
    Genmodel = Sequential()
    Genmodel.add(Dense(32,activation = 'linear', kernel_initializer = 'he_uniform', input_dim = infeats))
    Genmodel.add(Dense(32,activation = 'relu', kernel_initializer = 'he_uniform'))    
    Genmodel.add(Dense(64,activation = 'elu', kernel_initializer = 'he_uniform'))    
    Genmodel.add(Dense(32,activation = 'elu', kernel_initializer = 'he_uniform'))    
    Genmodel.add(Dense(32,activation = 'selu', kernel_initializer = 'he_uniform'))
    Genmodel.add(Dense(outfeats,activation = 'selu'))
    
    return Genmodel

#### 5. Define a function to create fake samples using the generator network
This function takes the random data distribution as input and generates the fake distribution.

In [5]:
# Function to create fake samples using the generator model
def fakedataGenerator(Genmodel, batch,infeats):
    ''' Function to create fake samples using the generator model. '''
    
    # First generate the inputs to the model
    genInputs = fakeInputs(batch,infeats)
    
    # Use these inputs inside the generator model to generate fake distribution
    X_fake = Genmodel.predict(genInputs)
    
    # Generate the labels of fake data set
    y_fake = np.zeros((batch,1))
    
    return X_fake, y_fake

#### 6. Define the global constants

In [6]:
# Define the arguments like batch size,input feature size and output feature size

batch    = 128
infeats  = 10
outfeats = 2

#### 7. Define a function to create the discriminator
This function is the same as the one we built in exercise **01.4_discriminator** to create a discriminator network.

In [ ]:
def discModel(outfeats):
    ''' Function which instantiates the discriminator model. '''
    Discmodel = Sequential()
    Discmodel.add(Dense(16, activation = 'relu',kernel_initializer = 'he_uniform',input_dim=outfeats))
    Discmodel.add(Dense(16,activation = 'relu' ,kernel_initializer = 'he_uniform'))
    Discmodel.add(Dense(16,activation = 'relu' ,kernel_initializer = 'he_uniform'))    
    Discmodel.add(Dense(1,activation = 'sigmoid'))
    
    # Compiling the model
    Discmodel.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])    
    return Discmodel 

#### 8. Print a summary of the discriminator network

In [8]:
# Print the summary of the discriminator model

Discmodel = discModel(outfeats)
Discmodel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                48        
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 609
Trainable params: 609
Non-trainable params: 0
_________________________________________________________________


#### 9. Invoke the generator model

In [9]:
# Instantiate the generator model by calling the genModel() function.

Genmodel = genModel(infeats, outfeats)
Genmodel.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 32)                352       
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_7 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_8 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 66        
Total params: 6,722
Trainable params: 6,722
Non-trainable params: 0
____________________________________________________

#### 10. Visualize the fake data distribution

In [ ]:
# Let's visualize the initial fake data
x_fake, _ = fakedataGenerator(Genmodel, batch, infeats)

# Plotting the fake data using pyplot
pyplot.scatter(x_fake[:, 0], x_fake[:, 1], color = 'blue')

# Adding x and y labels
pyplot.xlabel('Feature 1 of the distribution')
pyplot.ylabel('Feature 2 of the distribution')
pyplot.show()

The preceding plot shows that the data distribution is quite random. We can convert this random data into a form similar to the sine wave, which was our real data distribution. 

#### 11. Define a function to create the GAN model
Function ganModel is just a wrapper for the generator and discriminator models defined earlier.

In [ ]:
def ganModel(Genmodel,Discmodel):
    ''' Function which instantiates the GAN model.'''
    
    # First define that discriminator model cannot be trained
    Discmodel.trainable = False
    Ganmodel = Sequential()
    
    # First adding the generator model
    Ganmodel.add(Genmodel)
    
    # Next adding the discriminator model without training the parameters
    Ganmodel.add(Discmodel)
    
    # Compile the model for loss to optimise the Generator model
    Ganmodel.compile(loss='binary_crossentropy',optimizer = 'adam')  
    
    return Ganmodel

#### 12. Instantiate the GAN model

In [ ]:
# Initialise the gan model
gan_model = ganModel(Genmodel,Discmodel)

#### 13. Print summary of the GAN model

In [ ]:
# Print summary of the GAN model
gan_model.summary()

#### 14. Define the number of epochs

In [22]:
# Defining the number of epochs
nEpochs = 20000

#### 15. Train the GAN network

The following are the steps in the GAN model training process:
* 1. Generate a random number and then generate a batch of real samples and their labels.
* 2. Generate a batch of fake samples and their labels.
* 3. Train the discriminator model using the `train_on_batch()` function using the batch of real samples and fake samples.
* 4. Generate another batch of fake samples to train the GAN model.
* 5. Generate the labels for the fake samples meant to fool the discriminator.
* 6. Train the GAN model using the `train_on_batch()` function using the new fake samples and their labels.
* 7. Repeat step 1 to 6 for the number of epochs we want the training to run for. This can be done through a for loop over the number of epochs.
* 8. At every intermediate step, we calculate the accuracy of the model on the fake samples and real samples using the `evaluate()` function. We also generate output plots at certain epochs.

In [24]:
# Train the GAN network
for i in range(nEpochs):
    # Generate the random number for generating real samples
    loc = np.random.normal(3,1,1)
    
    # Generate samples equal to the bath size from the real distribution
    x_real, y_real = realData(loc,batch)
    
    # Generate fake samples using the fake data generator function
    x_fake, y_fake = fakedataGenerator(Genmodel,batch,infeats)
    
    # Train the  discriminator on the real samples
    Discmodel.train_on_batch(x_real, y_real)
    
    # Train the discriminator on the fake samples
    Discmodel.train_on_batch(x_fake, y_fake)
    
    # Generate new fake inputs for training the GAN network
    x_gan = fakeInputs(batch, infeats)
    
    # Create labels of the fake examples as 1 to fool the discriminator
    y_gan = np.ones((batch, 1))
    
    # Update the generator model through the discriminator model
    gan_model.train_on_batch(x_gan, y_gan)
    
    # Print the accuracy measures on the real and fake data for every 2000 epochs
    if (i) % 2000 == 0:
        # Generate samples equal to the bath size from the real distribution
        x_real, y_real = realData(loc,batch)
        
        # Evaluate Real distribution accuracy
        _, realAccuracy = Discmodel.evaluate(x_real, y_real, verbose = 0)
        
        # Generate fake samples using the fake data generator function
        x_fake,y_fake = fakedataGenerator(Genmodel,batch,infeats)
        
        # Evaluate fake distribution accuracy levels
        _, fakeAccuracy = Discmodel.evaluate(x_fake, y_fake, verbose = 0)
        
        print('Real accuracy:{R},Fake accuracy:{F}'.format(R = realAccuracy, F = fakeAccuracy))
        
        # Scatter plot real and fake data points
        pyplot.scatter(x_real[:, 0], x_real[:, 1], color = 'red')
        pyplot.scatter(x_fake[:, 0], x_fake[:, 1], color = 'blue')
        pyplot.xlabel('Feature 1 of the distribution')
        pyplot.ylabel('Feature 2 of the distribution')
        
        # Save plot to file
        filename = 'GAN_Training_Plot%03d.png' % (i)
        pyplot.savefig(filename)
        pyplot.close()

Real accuracy:0.0,Fake accuracy:0.984375
Real accuracy:1.0,Fake accuracy:0.9609375
Real accuracy:1.0,Fake accuracy:0.96875
Real accuracy:1.0,Fake accuracy:0.984375
Real accuracy:1.0,Fake accuracy:0.96875


#### Summary

This exercise presented the entire process of building a GAN.  We learned what a GAN is, its components, and how they  work together to generate a plausible data distribution. You now have the knowledge to create GANs using your own data.